In [23]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt

# Parámetros físicos
Omega = 3
Delta = 0
g = 0.95
gd = 0.05
ga = 0.015
num_traj = 100

# Parámetros del sistema
NF = 3
Kg = qt.basis(NF, 2)
Ka = qt.basis(NF, 1)
Ke = qt.basis(NF, 0)

# Operadores de proyección y transición
Agg = Kg * Kg.dag()
Aaa = Ka * Ka.dag()
Aee = Ke * Ke.dag()

Age = Kg * Ke.dag()  # σ-
Aeg = Age.dag()      # σ+

Aae = Ka * Ke.dag()
Aea = Aae.dag()
Aga = Kg * Ka.dag()
Aag = Aga.dag()

# Estado inicial
psi0 = Kg

# Tiempos
t_max = 50
dt = 0.01
steps = int(t_max / dt)
times = np.linspace(0, t_max, steps)



# Hamiltoniano (como antes)
H = Delta * Aee + 0.5 * Omega * (Aeg + Age)

# Operadores de colapso (decays)
c_ops = [
    np.sqrt(g) * Age,     # |e> → |g>
    np.sqrt(gd) * Aae,    # |e> → |a>
    np.sqrt(ga) * Aga     # |a> → |g>
]

# Operador de medición (heterodina típicamente usa σ-)
# Se necesita una lista de operadores medidos (en este caso sólo uno)
# También puedes medir múltiples observables
sc_ops = [np.sqrt(g) * Age]  # detector mide emisión de |e> → |g>

# Simulación con smesolve (trayectoria individual)

# Simulación con smesolve corregida
trajectories = qt.smesolve(
    H, psi0, times,
    c_ops=c_ops,
    sc_ops=sc_ops,
    e_ops=[Aee,Agg,Aee],
    ntraj=num_traj,
    heterodyne=True,           # <-- reemplaza "solver='heterodyne'"
    options={"store_measurement": True}              # <-- usa Options en vez de store_measurement directamente
)
# Extraer fotocorrientes y poblaciones promedio





10.0%. Run time:  20.29s. Est. time left: 00:00:03:02
20.0%. Run time:  43.04s. Est. time left: 00:00:02:52
30.0%. Run time:  65.61s. Est. time left: 00:00:02:33
40.0%. Run time:  88.14s. Est. time left: 00:00:02:12
50.0%. Run time: 110.80s. Est. time left: 00:00:01:50
60.0%. Run time: 133.38s. Est. time left: 00:00:01:28
70.0%. Run time: 155.68s. Est. time left: 00:00:01:06
80.0%. Run time: 178.44s. Est. time left: 00:00:00:44
90.0%. Run time: 201.40s. Est. time left: 00:00:00:22
100.0%. Run time: 225.04s. Est. time left: 00:00:00:00
Total run time: 227.58s


In [24]:
expect_values = [Aee,Agg,Aaa]

pop_e = trajectories.expect[0]

rho0 = qt.ket2dm(qt.basis(3, 0))
result_me = qt.mesolve(H_sys, rho0, times, c_ops,expect_values )
plt.plot(times, result_me.expect[1], 'k-', label='Lindblad', linewidth=2)

plt.xlabel("Tiempo")
plt.ylabel("Población $P_e(t)$")
plt.legend()
plt.show()

# Plots
plt.figure(figsize=(10, 6))
plt.plot(times, pop_e_avg, label='⟨e|ρ|e⟩')

plt.xlabel("Tiempo")
plt.ylabel("Población promedio")
plt.legend()
plt.title("Dinámica poblacional (promedio sobre trayectorias)")
plt.grid()
plt.show()


NameError: name 'H_sys' is not defined

In [ ]:
import numpy as np

# 1. Obtener los datos originales
I_t = np.array(trajectories.measurement)  # Shape: (100, 1, 2, 4999)

# 2. Eliminar la dimensión unitaria (100, 1, 2, 4999) -> (100, 2, 4999)
I_t = np.squeeze(I_t, axis=1)

# 3. Añadir un cero inicial a cada componente (real e imaginaria)
# Creamos arrays nuevos con 5000 puntos (4999 + 1 cero al inicio)
I_t_with_zero = np.zeros((100, 2, 5000))
I_t_with_zero[:, :, 1:] = I_t  # Los primeros 4999 puntos después del cero

# 4. Combinar partes real e imaginaria
trajectories_I = I_t_with_zero[:, 0, :] + 1j * I_t_with_zero[:, 1, :]  # Shape: (100, 5000)

# Verificación:
print("Shape final:", trajectories_I.shape)  # Debería ser (100, 5000)
print("\nPrimera trayectoria (primeros 5 puntos complejos):")
print(trajectories_I[0, :5])
print("\nÚltima trayectoria (últimos 5 puntos complejos):")
print(trajectories_I[-1, -5:])

In [ ]:
from scipy.signal import correlate
from scipy.fft import fft, fftfreq, fftshift
import matplotlib.pyplot as plt

num_traj, steps = trajectories_I.shape

# Autocorrelación promedio
autocorr_total = np.zeros(2*steps - 1, dtype=np.complex128)

for k in range(num_traj):
    I = trajectories_I[k, :]  # Fotocorriente compleja
    autocorr = correlate(I, I, mode='full')  # autocorrelación cruzada
    autocorr_total += autocorr

# Promedio
autocorr_avg = autocorr_total / num_traj

# Tiempo de retardo asociado
lags = np.arange(-steps + 1, steps) * dt
positive_lags = lags >= 0
tau = lags[positive_lags]
autocorr_pos = autocorr_avg[positive_lags] 
# Transformada de Fourier: S(ω) = ∫ dτ e^{iωτ} <I(t+τ) I(t)>
S_w = fftshift(fft(autocorr_pos))
freqs = fftshift(fftfreq(len(tau), dt))

In [ ]:
# Graficar
plt.figure(figsize=(8, 4))
plt.plot(freqs, np.real(S_w) / np.max(np.real(S_w)))
plt.xlabel("Frecuencia [ω]")
plt.ylabel("Espectro S(ω)")
plt.title("Espectro desde fotocorriente simulada")
plt.grid(True)
plt.axvline(x=Omega/(2*np.pi), color='r', linestyle='--', label=f'ω=±{Omega}')
plt.axvline(x=-Omega/(2*np.pi), color='r', linestyle='--')

plt.xlim(-3,3)
plt.show()